# タイタニックの生存予測  
https://signate.jp/competitions/102/

## ライブラリのインストール

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import optuna.integration.lightgbm as lgb_o
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

## データの読み込み

In [2]:
train = pd.read_csv("train.tsv", sep="\t", index_col=0) # 学習用データ
test = pd.read_csv("test.tsv", sep="\t", index_col=0)   # 評価用データ
sample_submit = pd.read_csv("sample_submit.tsv", sep="\t", index_col=0, header=None) # 応募用サンプルファイル

In [3]:
#skleanのver次第でLabelEncoderでエラーが出る可能性があるため追加
train.dropna(subset=["embarked"], inplace=True)
#train = train[~train["embarked"].isnull()]

## データの前処理

In [4]:
le = LabelEncoder()
train["sex"] = le.fit_transform(train["sex"])
le.classes_

array(['female', 'male'], dtype=object)

In [5]:
le = LabelEncoder()
train["embarked"] = le.fit_transform(train["embarked"])
le.classes_

array(['C', 'Q', 'S'], dtype=object)

## 年齢の情報を追加

In [6]:
train["age"].mean()

29.0954469273743

In [7]:
train["age"] = train["age"].fillna(train["age"].mean())

## 年齢層の情報を追加

In [8]:
bins = [0, 20, 40, 60, 80]
train["age_label"] = pd.cut(train["age"], bins=bins, labels=False)
train

,survived,pclass,sex,age,sibsp,parch,fare,embarked,age_label
id,,,,,,,,,
3,1,1,0,35.000000,1,0,53.1000,2,1
4,0,3,1,35.000000,0,0,8.0500,2,1
7,0,3,1,2.000000,3,1,21.0750,2,0
9,1,2,0,14.000000,1,0,30.0708,0,0
11,1,1,0,58.000000,0,0,26.5500,2,2
...,...,...,...,...,...,...,...,...,...
873,0,3,1,47.000000,0,0,9.0000,2,2
874,1,2,0,28.000000,1,0,24.0000,0,1
879,1,1,0,56.000000,0,1,83.1583,0,2


## 家族の情報を追加

In [9]:
train["family"] = train["sibsp"] + train["parch"] +1
train

,survived,pclass,sex,age,sibsp,parch,fare,embarked,age_label,family
id,,,,,,,,,,
3,1,1,0,35.000000,1,0,53.1000,2,1,2
4,0,3,1,35.000000,0,0,8.0500,2,1,1
7,0,3,1,2.000000,3,1,21.0750,2,0,5
9,1,2,0,14.000000,1,0,30.0708,0,0,2
11,1,1,0,58.000000,0,0,26.5500,2,2,1
...,...,...,...,...,...,...,...,...,...,...
873,0,3,1,47.000000,0,0,9.0000,2,2,1
874,1,2,0,28.000000,1,0,24.0000,0,1,2
879,1,1,0,56.000000,0,1,83.1583,0,2,2


## LightGBMで作成

https://smart-hint.com/ml/gbdt-image/

In [10]:
data = train.drop(["survived"], axis=1)
target = train["survived"]

In [11]:
params = {
    "metric": "auc",
    "objective": "binary",
    #'learning_rate': 0.01,
    'min_child_samples': 5,
    #'force_col_wise': True
}

In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=1) 

In [13]:
auc_ave = []

for train_index, valid_index in kf.split(data):
    
    X_train = data.iloc[train_index]
    X_valid = data.iloc[valid_index]
    
    y_train = target.iloc[train_index]
    y_valid = target.iloc[valid_index]
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)
    
    lgbm = lgb.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=1000,
        early_stopping_rounds=50,
        #verbose_eval=False
    )
    
    #y_train_pred = lgbm.predict(x_train)
    y_pred = lgbm.predict(X_valid)
    
    auc = roc_auc_score(y_valid, y_pred)
    auc_ave.append(auc)
    
    print(auc)

print("ave:{:.3f}".format(np.mean(auc_ave)))

[LightGBM] [Info] Number of positive: 143, number of negative: 211
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000536 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 158
[LightGBM] [Info] Number of data points in the train set: 354, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.403955 -> initscore=-0.389014
[LightGBM] [Info] Start training from score -0.389014
[1]	valid_0's auc: 0.910428
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.912567
[3]	valid_0's auc: 0.916043
[4]	valid_0's auc: 0.908556
[5]	valid_0's auc: 0.912032
[6]	valid_0's auc: 0.91016
[7]	valid_0's auc: 0.923529
[8]	valid_0's auc: 0.924064
[9]	valid_0's auc: 0.924064
[10]	valid_0's auc: 0.925936
[11]	valid_0's auc: 0.926471
[12]	valid_0's auc: 0.924866
[13]	valid_0's auc: 0.926471
[14]	valid_0's auc:

[72]	valid_0's auc: 0.87035
[73]	valid_0's auc: 0.869811
[74]	valid_0's auc: 0.869811
[75]	valid_0's auc: 0.87035
[76]	valid_0's auc: 0.872507
[77]	valid_0's auc: 0.871429
[78]	valid_0's auc: 0.871429
[79]	valid_0's auc: 0.873046
[80]	valid_0's auc: 0.873046
[81]	valid_0's auc: 0.873046
[82]	valid_0's auc: 0.873585
[83]	valid_0's auc: 0.873585
[84]	valid_0's auc: 0.873585
[85]	valid_0's auc: 0.873046
[86]	valid_0's auc: 0.872507
[87]	valid_0's auc: 0.871968
[88]	valid_0's auc: 0.871429
[89]	valid_0's auc: 0.873046
[90]	valid_0's auc: 0.873585
[91]	valid_0's auc: 0.873585
[92]	valid_0's auc: 0.875202
[93]	valid_0's auc: 0.875741
[94]	valid_0's auc: 0.875202
[95]	valid_0's auc: 0.873585
[96]	valid_0's auc: 0.873046
[97]	valid_0's auc: 0.873046
[98]	valid_0's auc: 0.872507
[99]	valid_0's auc: 0.873046
[100]	valid_0's auc: 0.873585
[101]	valid_0's auc: 0.873046
[102]	valid_0's auc: 0.873046
[103]	valid_0's auc: 0.871968
[104]	valid_0's auc: 0.873046
Early stopping, best iteration is:
[54]	

## LightGBMのハイパーパラメータ調整

In [14]:
params = {
    "metric": "auc",
    "objective": "binary",
    #'force_col_wise': True
}

In [15]:
auc_ave = []
best_params = []
models = []

for train_index, valid_index in kf.split(data):
    
    X_train = data.iloc[train_index]
    X_valid = data.iloc[valid_index]
    
    y_train = target.iloc[train_index]
    y_valid = target.iloc[valid_index]
    
    lgb_train = lgb_o.Dataset(X_train, y_train)
    lgb_eval = lgb_o.Dataset(X_valid, y_valid)
    
    lgbm = lgb_o.train(
        params,
        lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=1000,
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    #y_train_pred = lgbm.predict(x_train)
    y_pred = lgbm.predict(X_valid)
    
    auc = roc_auc_score(y_valid, y_pred)
    auc_ave.append(auc)
    
    best_params.append(lgbm.params)
    
    models.append(lgbm)
    
    print(auc)

print("ave:{:.3f}".format(np.mean(auc_ave)))

[I 2024-01-18 13:27:26,465] A new study created in memory with name: no-name-6c300236-5232-4bcf-9c29-c31f80df8cfb
feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 143, number of negative: 211
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 158
[LightGBM] [Info] Number of data points in the train set: 354, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.403955 -> initscore=-0.389014
[LightGBM] [Info] Start training from score -0.389014
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

feature_fraction, val_score: 0.948128:  43%|###################2                         | 3/7 [00:00<00:00, 23.19it/s]

[LightGBM] [Info] Number of positive: 143, number of negative: 211
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000143 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 158
[LightGBM] [Info] Number of data points in the train set: 354, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.403955 -> initscore=-0.389014
[LightGBM] [Info] Start training from score -0.389014
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[I 2024-01-18 13:27:26,662] Trial 3 finished with value: 0.9454545454545454 and parameters: {'feature_fraction': 0.4}. Best is trial 1 with value: 0.9481283422459893.
feature_fraction, val_score: 0.948128:  71%|################################1            | 5/7 [00:00<00:00, 23.19it/s]

...

min_data_in_leaf, val_score: 0.803857:  60%|###########################                  | 3/5 [00:00<00:00, 16.26it/s]

[LightGBM] [Info] Number of positive: 144, number of negative: 211
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000098 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 157
[LightGBM] [Info] Number of data points in the train set: 355, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.405634 -> initscore=-0.382045
[LightGBM] [Info] Start training from score -0.382045
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

min_data_in_leaf, val_score: 0.803857: 100%|#############################################| 5/5 [00:00<00:00, 18.88it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [16]:
best_params

[{'metric': 'auc',
  'objective': 'binary',
  'feature_pre_filter': False,
  'lambda_l1': 0.014955489930001849,
  'lambda_l2': 2.2910008571358935e-06,
  'num_leaves': 11,
  'feature_fraction': 0.7,
  'bagging_fraction': 0.7008070212902303,
  'bagging_freq': 5,
  'min_child_samples': 20,
  'num_iterations': 1000,
  'early_stopping_round': 50},
 {'metric': 'auc',
  'objective': 'binary',
  'feature_pre_filter': False,
  'lambda_l1': 0.03534583683603272,
  'lambda_l2': 0.004143658996054168,
  'num_leaves': 31,
  'feature_fraction': 0.8999999999999999,
  'bagging_fraction': 0.8849505214185354,
  'bagging_freq': 7,
  'min_child_samples': 20,
  'num_iterations': 1000,
  'early_stopping_round': 50},
 {'metric': 'auc',
  'objective': 'binary',
  'feature_pre_filter': False,
  'lambda_l1': 0.0,
  'lambda_l2': 0.0,
  'num_leaves': 31,
  'feature_fraction': 0.6,
  'bagging_fraction': 1.0,
  'bagging_freq': 0,
  'min_child_samples': 20,
  'num_iterations': 1000,
  'early_stopping_round': 50},
 {'m

## 提出データの作成

In [17]:
le = LabelEncoder()
test["sex"] = le.fit_transform(test["sex"])
le.classes_

array(['female', 'male'], dtype=object)

In [18]:
le = LabelEncoder()
test["embarked"] = le.fit_transform(test["embarked"])
le.classes_

array(['C', 'Q', 'S'], dtype=object)

In [19]:
test["age"] = test["age"].fillna(train["age"].mean())

In [20]:
test["age_label"] = pd.cut(test["age"], bins=bins, labels=False)

In [21]:
test["family"] = test["sibsp"] + test["parch"] +1

In [22]:
test

,pclass,sex,age,sibsp,parch,fare,embarked,age_label,family
id,,,,,,,,,
0,3,1,22.000000,1,0,7.2500,2,1,2
1,1,0,38.000000,1,0,71.2833,0,1,2
2,3,0,26.000000,0,0,7.9250,2,1,1
5,3,1,29.095447,0,0,8.4583,1,1,1
6,1,1,54.000000,0,0,51.8625,2,2,1
...,...,...,...,...,...,...,...,...,...
885,3,0,39.000000,0,5,29.1250,1,1,6
886,2,1,27.000000,0,0,13.0000,2,1,1
887,1,0,19.000000,0,0,30.0000,2,0,1


In [23]:
i = 0
df_submit = pd.DataFrame([])
for m in models:
    y_submit = m.predict(test)
    df_submit[i] = y_submit
    i+=1

In [24]:
df_submit

,0,1,2,3,4
0,0.109581,0.060002,0.030493,0.307286,0.023982
1,0.926178,0.975642,0.998223,0.545786,0.991446
2,0.574585,0.573191,0.512331,0.439533,0.598901
3,0.240236,0.306811,0.077071,0.364926,0.758567
4,0.325768,0.324322,0.241350,0.393813,0.298218
...,...,...,...,...,...
441,0.376733,0.288000,0.161493,0.467680,0.198110
442,0.216339,0.162651,0.115171,0.364926,0.078614
443,0.809464,0.937089,0.989740,0.552317,0.985746
444,0.511111,0.520826,0.715825,0.393813,0.786677


In [25]:
df_submit["ave"] = df_submit.mean(axis=1)
df_submit.head()

,0,1,2,3,4,ave
0,0.109581,0.060002,0.030493,0.307286,0.023982,0.106269
1,0.926178,0.975642,0.998223,0.545786,0.991446,0.887455
2,0.574585,0.573191,0.512331,0.439533,0.598901,0.539708
3,0.240236,0.306811,0.077071,0.364926,0.758567,0.349522
4,0.325768,0.324322,0.241350,0.393813,0.298218,0.316694


In [26]:
sample_submit

,1
0,
0,0
1,1
2,0
5,1
6,1
...,...
885,1
886,1
887,1


In [27]:
df_submit.index = sample_submit.index
df_submit

,0,1,2,3,4,ave
0,,,,,,
0,0.109581,0.060002,0.030493,0.307286,0.023982,0.106269
1,0.926178,0.975642,0.998223,0.545786,0.991446,0.887455
2,0.574585,0.573191,0.512331,0.439533,0.598901,0.539708
5,0.240236,0.306811,0.077071,0.364926,0.758567,0.349522
6,0.325768,0.324322,0.241350,0.393813,0.298218,0.316694
...,...,...,...,...,...,...
885,0.376733,0.288000,0.161493,0.467680,0.198110,0.298403
886,0.216339,0.162651,0.115171,0.364926,0.078614,0.187540
887,0.809464,0.937089,0.989740,0.552317,0.985746,0.854871


In [28]:
df_submit["ave"].to_csv('submit_lightgbm_model.tsv', header=None, sep='\t')